In [3]:
import geopandas as gpd
from shapely.geometry import box
import re
import os

# === 1. 定义输入输出路径 === #
shp_folder = r'D:\file\a_clip_data\c_merge'
dsm_folder = r'D:\file\d_height'
output_file = r'D:\file\match_results.txt'  # 存储匹配结果的 txt 文件

shp_files = [f for f in os.listdir(shp_folder) if f.endswith('.shp')]
dsm_files = [f for f in os.listdir(dsm_folder) if f.endswith('_DEM.tif')]

# 提取经纬度的函数
pattern = re.compile(r'Copernicus_DSM_10_([NS])(\d+)_00_([EW])(\d+)_00_DEM')

def get_bbox_from_name(name):
    """从 DSM 文件名提取边界框"""
    match = pattern.search(name)
    if match:
        ns, lat, ew, lon = match.groups()
        lat = int(lat) * (1 if ns == 'N' else -1)
        lon = int(lon) * (1 if ew == 'E' else -1)
        return name, box(lon, lat, lon + 1, lat + 1)
    return None

# 创建 DSM 文件的边界框字典
dsm_bboxes = dict(filter(None, [get_bbox_from_name(f) for f in dsm_files]))

# === 2. 处理每个 shapefile 并写入结果文件 === #
with open(output_file, 'w', encoding='utf-8') as f_out:
    for shp_name in shp_files:
        shp_path = os.path.join(shp_folder, shp_name)

        # 读取 shapefile
        gdf = gpd.read_file(shp_path)

        # 过滤空 geometry
        gdf = gdf[gdf.geometry.notna()]
        gdf = gdf[~gdf.geometry.is_empty]

        # 计算 polygon 的 bbox
        gdf_bounds = gdf.geometry.bounds
        gdf['bbox_geom'] = gdf_bounds.apply(lambda row: box(row.minx, row.miny, row.maxx, row.maxy), axis=1)

        # === 3. 匹配 polygon bbox 和 DSM bbox === #
        gdf['intersecting_dsm_files'] = gdf['bbox_geom'].apply(
            lambda poly_box: [fname for fname, dsm_box in dsm_bboxes.items() if poly_box.intersects(dsm_box)]
        )

        # === 4. 将结果写入文件 === #
        #f_out.write(f"\n=== {shp_name} 匹配结果 ===\n")
        for idx, row in gdf.iterrows():
            matched_files = ", ".join(row['intersecting_dsm_files']) if row['intersecting_dsm_files'] else "无匹配"
            #f_out.write(f"Polygon {idx} 匹配到 DSM 文件: {matched_files}\n")
            f_out.write(f"{matched_files}\n")
            print(f"{matched_files}\n")

print(f"匹配结果已保存到 {output_file}")

Copernicus_DSM_10_N45_00_E004_00_DEM.tif, Copernicus_DSM_10_N45_00_E005_00_DEM.tif, Copernicus_DSM_10_N46_00_E004_00_DEM.tif, Copernicus_DSM_10_N46_00_E005_00_DEM.tif

Copernicus_DSM_10_N45_00_E015_00_DEM.tif, Copernicus_DSM_10_N45_00_E016_00_DEM.tif

Copernicus_DSM_10_N44_00_E033_00_DEM.tif, Copernicus_DSM_10_N44_00_E034_00_DEM.tif, Copernicus_DSM_10_N45_00_E033_00_DEM.tif, Copernicus_DSM_10_N45_00_E034_00_DEM.tif

Copernicus_DSM_10_N33_00_W078_00_DEM.tif, Copernicus_DSM_10_N33_00_W079_00_DEM.tif

Copernicus_DSM_10_N33_00_E130_00_DEM.tif, Copernicus_DSM_10_N33_00_E131_00_DEM.tif, Copernicus_DSM_10_N34_00_E130_00_DEM.tif, Copernicus_DSM_10_N34_00_E131_00_DEM.tif

Copernicus_DSM_10_N33_00_E132_00_DEM.tif

Copernicus_DSM_10_N33_00_W099_00_DEM.tif, Copernicus_DSM_10_N34_00_W099_00_DEM.tif

Copernicus_DSM_10_N33_00_W079_00_DEM.tif, Copernicus_DSM_10_N33_00_W080_00_DEM.tif, Copernicus_DSM_10_N34_00_W079_00_DEM.tif, Copernicus_DSM_10_N34_00_W080_00_DEM.tif

Copernicus_DSM_10_N33_00_E130_00_D

In [4]:
file_path = r'D:\file\match_results.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    line_count = sum(1 for _ in f)

print(f"文件共有 {line_count} 行")

文件共有 269 行


In [5]:
import os
import rasterio
from rasterio.merge import merge

# === 1. 定义路径 === #
match_results_file = r'D:\file\match_results.txt'
dsm_folder = r'D:\file\d_height_filter'
output_base_folder = r'D:\file\d_impervious_surface'

# 确保输出文件夹存在
os.makedirs(output_base_folder, exist_ok=True)

# === 2. 读取匹配的 DSM 文件名 === #
with open(match_results_file, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]  # 去除空行

# === 3. 逐行合并 TIFF 并存储 === #
for i, line in enumerate(lines):
    tiff_files = [os.path.join(dsm_folder, f.strip()) for f in line.split(",") if f.strip()]

    if not tiff_files:
        print(f"跳过第 {i} 组, 没有找到匹配文件")
        continue

    output_folder = os.path.join(output_base_folder, str(i))
    os.makedirs(output_folder, exist_ok=True)  # 创建对应的输出目录
    output_tiff = os.path.join(output_folder, f"merged_{i}.tif")

    # 读取 TIFF 并合并
    src_files = []
    for file in tiff_files:
        if os.path.exists(file):  # 确保文件存在
            src = rasterio.open(file)
            src_files.append(src)
        else:
            print(f"警告: 文件 {file} 不存在，跳过")

    if not src_files:
        print(f"跳过第 {i} 组, 没有可用的 TIFF 文件")
        continue

    # 合并影像
    mosaic, out_trans = merge(src_files)

    # 复制元数据
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # 写入合并后的 TIFF
    with rasterio.open(output_tiff, "w", **out_meta) as dest:
        dest.write(mosaic)

    # 关闭所有打开的 TIFF 文件
    for src in src_files:
        src.close()

    print(f"合并完成: {output_tiff}")

print("所有 TIFF 影像合并完成！")


合并完成: D:\file\d_buildup_isf\0\merged_0.tif
合并完成: D:\file\d_buildup_isf\1\merged_1.tif
合并完成: D:\file\d_buildup_isf\2\merged_2.tif
合并完成: D:\file\d_buildup_isf\3\merged_3.tif
合并完成: D:\file\d_buildup_isf\4\merged_4.tif
合并完成: D:\file\d_buildup_isf\5\merged_5.tif
合并完成: D:\file\d_buildup_isf\6\merged_6.tif
合并完成: D:\file\d_buildup_isf\7\merged_7.tif
合并完成: D:\file\d_buildup_isf\8\merged_8.tif
合并完成: D:\file\d_buildup_isf\9\merged_9.tif
合并完成: D:\file\d_buildup_isf\10\merged_10.tif
合并完成: D:\file\d_buildup_isf\11\merged_11.tif
合并完成: D:\file\d_buildup_isf\12\merged_12.tif
合并完成: D:\file\d_buildup_isf\13\merged_13.tif
合并完成: D:\file\d_buildup_isf\14\merged_14.tif
合并完成: D:\file\d_buildup_isf\15\merged_15.tif
合并完成: D:\file\d_buildup_isf\16\merged_16.tif
合并完成: D:\file\d_buildup_isf\17\merged_17.tif
合并完成: D:\file\d_buildup_isf\18\merged_18.tif
合并完成: D:\file\d_buildup_isf\19\merged_19.tif
合并完成: D:\file\d_buildup_isf\20\merged_20.tif


KeyboardInterrupt: 